In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
! /opt/bin/nvidia-smi

In [ ]:
 (x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
y_train[0]

In [ ]:
x_train[0]

In [ ]:
#畫出MNIST圖片

import matplotlib.pyplot as plt

plt.gcf().set_size_inches(15,4)
for i in range(5):
  ax = plt.subplot(1,5,i+1)
  ax.imshow(x_train[i],cmap='gray') #cmap='binary'會白底黑字
  ax.set_title('label='+str(y_train[i]),fontsize=18)
plt.show()

In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
x_train[0]

In [ ]:
x_train.shape

In [ ]:
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000,28,28,1))

In [ ]:
x_train.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
CNN = keras.Sequential(name = 'CNNNetwork') #model名稱:CNN

兩次convalution兩次pooling

In [ ]:
CNN.add(layers.Conv2D(filters=32, 
                      kernel_size=(3,3),
                      activation='relu',
                      padding='same',
                      input_shape=(28, 28, 1) #第一層才要給
        ))

In [ ]:
keras.utils.plot_model(CNN, show_shapes=True)

In [ ]:
CNN.add(layers.MaxPooling2D(2,2))

In [ ]:
keras.utils.plot_model(CNN,show_shapes=True)

In [ ]:
CNN.add(layers.Conv2D(filters=64,
                           kernel_size=(3,3),
                           #padding='same', #故意不加
                           activation='relu'))
CNN.add(layers.MaxPooling2D(2,2))

In [ ]:
keras.utils.plot_model(CNN,show_shapes=True)

In [ ]:
print(CNN.summary())

 - 320 = ((3x3x1)+1)x32
 - 3x3 : filter size
 - x1 (28x28x1)
 - +1 bias
 - 32個filter


18496 = ((3x3x32)+1)x64
 - 3x3 : filter size
 - 32 : 14x14x32 
 - +1 bias
 - 64 filter



In [ ]:
CNN.add(layers.Flatten())
CNN.add(layers.Dense(128,activation='relu'))
CNN.add(layers.Dense(64,activation='relu'))
CNN.add(layers.Dense(10,activation='softmax')) #最後一層,10:配合資料類別數量 0,1,2,3,4,5,6,7,8,9


In [ ]:
keras.utils.plot_model(CNN,show_shapes=True)

In [ ]:
print(CNN.summary())

 #參數解釋
 - 2304 = 6x6x64
 - 295040 = (2304+1)x128
 - 8256 = (128+1)x64
 - 650 = (64+1)x10

In [ ]:
CNN.compile(optimizer='Adam', 
            loss=keras.losses.sparse_categorical_crossentropy, 
            metrics=['accuracy'])

In [ ]:
train_history = CNN.fit(x=x_train,
                        y=y_train,
                        validation_split=0.2, #分2/8分
                        epochs=20, #訓練20次
                        batch_size=300, #跑300張之後更新一次weight
                        verbose=2 #訓練的狀況顯示方式
                        )

In [ ]:
#分析訓練結果圖
import matplotlib.pyplot as plt
def show_train_history(show_train_value,test_value):
    plt.plot(train_history.history[show_train_value])
    plt.plot(train_history.history[test_value])
    plt.title('Train History')
    plt.ylabel(show_train_value)
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
show_train_history('accuracy','val_accuracy')
show_train_history('loss','val_loss')

In [ ]:
import numpy as np
np.mean(CNN.predict_classes(x_test)==y_test)

In [ ]:
prediction = np.argmax(CNN.predict(x_test), axis = -1)

In [ ]:
CNN.predict(x_test).shape

In [ ]:
import pandas as pd

In [ ]:
pd.crosstab(y_test,prediction,rownames=['label'],colnames=['predict'])

In [ ]:
#畫出來比對用的
import matplotlib.pyplot as plt
def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14)
    if num>25: num=25
    images = images.reshape(images.shape[0],28,28)
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[idx], cmap='binary')

        ax.set_title("label=" +str(labels[idx])+
                     ",predict="+str(prediction[idx])
                     ,fontsize=10,color='red') 
        
        ax.set_xticks([]);ax.set_yticks([])        
        idx+=1 
    plt.show()

In [ ]:
plot_images_labels_prediction(x_test,y_test,prediction,idx=10)

找出預測錯誤問題

In [ ]:
df = pd.DataFrame({
    'label':y_test,
    'predict':prediction
})

In [ ]:
df.head()

In [ ]:
df[ (df.label == 4) & (df.predict == 9) ]


In [ ]:
plot_images_labels_prediction(x_test,y_test,prediction,idx=2130)

In [ ]:
CNN.save('MyMinistModel.h5')

In [ ]:
!pwd

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('MyMinistModel.h5')

In [ ]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_test.shape

In [ ]:
x_test = x_test/255
x_test = x_test.reshape(10000, 28, 28, 1)


In [ ]:
model.evaluate(x_test,y_test)[1]